In [1]:
from pathlib import Path
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

Using TensorFlow backend.


In [39]:
f = Path('Models/Standard/V2/M5/model_structure5.json')
model_structure = f.read_text()
model = model_from_json(model_structure)
model.load_weights('Models/Standard/V2/M5/model_weights(M5E6).h5')
model.summary()
model.compile(
    loss='binary_crossentropy',
    optimizer='rmsprop', 
    metrics=['accuracy']
)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 256, 256, 16)      2320      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
batch_normalization_16 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 64, 64, 32)       

In [40]:
gen = ImageDataGenerator()
test_gen = gen.flow_from_directory('chest_xray/test/',
                                       target_size=(256, 256),
                                       batch_size= 64,
                                       class_mode='binary', 
                                       shuffle=False)
test_steps_per_epoch = np.math.ceil(test_gen.samples / test_gen.batch_size)

Found 624 images belonging to 2 classes.


In [46]:
Y_pred = model.predict_generator(test_gen, test_steps_per_epoch)
y_pred = np.argmax(Y_pred, axis=1)
y_test=np.zeros(Y_pred.size)
for i in range(Y_pred.size):
     if (Y_pred[i]>=0.5):
         y_test[i]=1

In [47]:
print('Confusion Matrix')
print(confusion_matrix(test_gen.classes, y_test))
print('Classification Report')
target_names = ['NORMAL', 'PNEUMONIA']
print(classification_report(test_gen.classes, y_test, 
target_names=target_names))

Confusion Matrix
[[145  89]
 [ 10 380]]
Classification Report
              precision    recall  f1-score   support

      NORMAL       0.94      0.62      0.75       234
   PNEUMONIA       0.81      0.97      0.88       390

    accuracy                           0.84       624
   macro avg       0.87      0.80      0.82       624
weighted avg       0.86      0.84      0.83       624



In [28]:
img= image.load_img('test.jpg', target_size=(256,256))
image_to_test = image.img_to_array(img) / 255

Since Keras expects a list of images, not a single image, we add a fourth dimension to the image's array object


In [29]:
list_of_images = np.expand_dims(image_to_test, axis=0)


Now we make a prediction using the model
- Results contains the list of results for images we pass in.
- Since we are only testing one image, we only need to check the first result.


In [32]:
results = model.predict(list_of_images)  
single_result = results[0]

[1.]


We will get a likelihood score for both possible classifications 
- Now we can find out which class had the highest score.

In [20]:
labels = ['Normal', 'Pneumonia']
most_likely_class_index = int(np.argmax(single_result))
class_likelihood = single_result [most_likely_class_index]
label = labels[most_likely_class_index]
print("This is image is a {} - Likelihood: {:2f}".format(label, class_likelihood))

This is image is a Normal - Likelihood: 0.988434
